In [2]:
import ingest
import pandas as pd
pd.set_option('display.max_columns', None)

import altair as alt
alt.data_transformers.disable_max_rows()

import ipywidgets
from ipywidgets import interact

import numpy as np
from sklearn.preprocessing import OneHotEncoder

from tqdm import tqdm

In [3]:
outpatient = ingest.get_cache_data("Outpatient", "~/Downloads/outpatient.pkl")

INFO:root:Reading local cache file C:\Users\Taylor School\Downloads\outpatient.pkl


In [4]:
outpatient = outpatient.dropna(how='all', subset=['HCPCS_CD_1', 'HCPCS_CD_2','HCPCS_CD_3','HCPCS_CD_4','HCPCS_CD_5','HCPCS_CD_6','HCPCS_CD_7',
'HCPCS_CD_8','HCPCS_CD_9','HCPCS_CD_10','HCPCS_CD_11','HCPCS_CD_12','HCPCS_CD_13','HCPCS_CD_14','HCPCS_CD_15',
'HCPCS_CD_16','HCPCS_CD_17','HCPCS_CD_18','HCPCS_CD_19','HCPCS_CD_20','HCPCS_CD_21','HCPCS_CD_22','HCPCS_CD_23',
'HCPCS_CD_24','HCPCS_CD_25','HCPCS_CD_26','HCPCS_CD_27','HCPCS_CD_28','HCPCS_CD_29','HCPCS_CD_30','HCPCS_CD_31',
'HCPCS_CD_32','HCPCS_CD_33','HCPCS_CD_34','HCPCS_CD_35','HCPCS_CD_36','HCPCS_CD_37','HCPCS_CD_38','HCPCS_CD_39',
'HCPCS_CD_40','HCPCS_CD_41','HCPCS_CD_42','HCPCS_CD_43','HCPCS_CD_44','HCPCS_CD_45'])

In [5]:
xy = outpatient.set_index(['CLM_ID', 'SEGMENT'])

outpatient = None

x = xy[['HCPCS_CD_1', 'HCPCS_CD_2','HCPCS_CD_3','HCPCS_CD_4','HCPCS_CD_5','HCPCS_CD_6','HCPCS_CD_7',
'HCPCS_CD_8','HCPCS_CD_9','HCPCS_CD_10','HCPCS_CD_11','HCPCS_CD_12','HCPCS_CD_13','HCPCS_CD_14','HCPCS_CD_15',
'HCPCS_CD_16','HCPCS_CD_17','HCPCS_CD_18','HCPCS_CD_19','HCPCS_CD_20','HCPCS_CD_21','HCPCS_CD_22','HCPCS_CD_23',
'HCPCS_CD_24','HCPCS_CD_25','HCPCS_CD_26','HCPCS_CD_27','HCPCS_CD_28','HCPCS_CD_29','HCPCS_CD_30','HCPCS_CD_31',
'HCPCS_CD_32','HCPCS_CD_33','HCPCS_CD_34','HCPCS_CD_35','HCPCS_CD_36','HCPCS_CD_37','HCPCS_CD_38','HCPCS_CD_39',
'HCPCS_CD_40','HCPCS_CD_41','HCPCS_CD_42','HCPCS_CD_43','HCPCS_CD_44','HCPCS_CD_45']]

y = ['CLM_PMT_AMT']

xy = None

In [6]:
def encode_one_column(one_col_df, col):
    one_col_df.dropna(inplace=True)
    one_col_df.rename(columns = {col:'HCPCS_CD'}, inplace=True)
    one_col_df['HCPCS_CD'] = one_col_df['HCPCS_CD'][:3]
    return pd.get_dummies(one_col_df, sparse=True)


In [7]:
encoded_x = pd.DataFrame()

for col in tqdm(x.columns.tolist()):
    encoded_x =encoded_x.append(encode_one_column(x[col].to_frame(), col), ignore_index=False)

 56%|█████▌    | 25/45 [32:57<26:21, 79.09s/it] 


MemoryError: Unable to allocate 261. MiB for an array with shape (68539763,) and data type int32

In [ ]:
encoded_x = encoded_x.fillna(0)

tqdm.pandas()
encoded_x = encoded_x.groupby(level=['CLM_ID', 'SEGMENT'], sort=False, observed=True).progress_aggregate(sum)


  3%|▎         | 43520/1507622 [05:49<3:15:57, 124.53it/s]


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

X_train, X_test, y_train, y_test = train_test_split(encoded_x, y, test_size=0.4, random_state=0)
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
reg.score(X_test, y_test)